In [103]:
from sendkeys import PressKey,ReleaseKey
import time
import cv2
import imutils
from imutils.object_detection import non_max_suppression
from PIL import ImageGrab
import numpy as np
import math

In [104]:
#A function to show image
def show_img(image):
    while True:
        cv2.imshow('Image window',image)
        if cv2.waitKey(10) & 0XFF==ord('q'):
            cv2.destroyAllWindows()
            break
        

In [105]:
#Pedestrian detection using HOG 

hog=cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def pedestrian_detection(image):
    gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects,weights=hog.detectMultiScale(gray_image,winStride=(4,4),padding=(4,4),scale=2)
    rects=np.array([[x,y,x+w,y+h] for x,y,w,h in rects])    
    final_rects=non_max_suppression(rects,overlapThresh=0.5)
    for x,y,p,q in final_rects:    
        image=cv2.rectangle(image,(x,y),(p,q),(0,0,255),2)
    return image
    

In [106]:
#Haar cascacade method of pedestrian detection

people_detecter=cv2.CascadeClassifier('haarcascade_fullbody.xml')
def cascade_classifier(image):
    gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects=people_detecter.detectMultiScale(gray_image,1.05,3)
    rects=np.array([[x,y,x+w,y+h] for x,y,w,h in rects])    
    final_rects=non_max_suppression(rects,overlapThresh=0.5)
    for (x,y,w,h) in rects:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
    return image

In [107]:
#edge detection
def image_process(image):
    image=cv2.Canny(image,150,250)
    return image

#Region of interest
def ROI(image):
    roi=np.zeros_like(image)
    roi=cv2.fillPoly(roi,[vertices],255)
    image=cv2.bitwise_and(image,roi)
    lines=cv2.HoughLinesP(image,1,np.pi/180,110,np.array([]),minLineLength=120,maxLineGap=20)
    image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    try:
        if len(lines)>2:    
            lines=lines_sel(lines)
        for [[x,y,p,q]] in lines:
            cv2.line(image,(x,y),(p,q),(0,0,255),3)
    except:
        pass
    return image,lines



In [108]:
def forward():
    PressKey(0X39)
    time.sleep(1)
    ReleaseKey(0XCB)
    ReleaseKey(0XCD)
    ReleaseKey(0X39)

def right():
    PressKey(0XCD)
    PressKey(0X39)
    time.sleep(.5)
    ReleaseKey(0XCB)
    ReleaseKey(0XCD)
    ReleaseKey(0X39)
def left():
    PressKey(0XCB)
    PressKey(0X39)
    time.sleep(.5)
    ReleaseKey(0XCB)
    ReleaseKey(0XCD)
    ReleaseKey(0X39)

In [109]:
def lines_sel(lines):
    #make line equation
    dist=[]
    slopes=[]
    bias=[]
    lines_to_draw=[]
    checked=[]
    for[[x,y,p,q]] in lines:
        m=(q-y)/(p-x)
        b=y-m*x
        slopes.append(m)
        bias.append(b)
        dist.append(np.sqrt(np.power(p-x,2)+np.power(q-y,2)))

    while(max(dist)!=0):
        index=dist.index(max(dist))
        if 0.2<=abs(slopes[index])<=2.74:
            lines_to_draw.append(lines[index])
        dist[index]=0
        if len(lines_to_draw)==2:
            break
    return(lines_to_draw)       
    

In [110]:
def drive(lines_to_draw):
    try:
        slopes_to_draw=[]
        for [[x,y,p,q]] in lines_to_draw:
            slopes_to_draw.append((q-y)/(p-x))

        sign=slopes_to_draw[0]/abs(slopes_to_draw[0])
        if (slopes_to_draw[1]/abs(slopes_to_draw[1]))!=sign:
            forward()
        elif(sign==-1):
            right()
        elif(sign==1):
            left()    
    except:
        pass

In [114]:
time.sleep(5)
vertices=np.array([[-200,640],[400,450],[1000,640],[600,300],[200,300]],'int32')
#image grab and show
while(True):
    screen=np.array(ImageGrab.grab(bbox=(0,40,800,640)))
    #screen=cv2.resize(screen,(400,300))
    screen=image_process(screen)
    screen,lines=ROI(screen)
    drive(lines) 
    cv2.imshow('win',screen)
    if cv2.waitKey(25) & 0XFF==ord('q'):
        cv2.destroyAllWindows()
        break



C:\Users\Nahar\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in int_scalars
  if __name__ == '__main__':
C:\Users\Nahar\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
C:\Users\Nahar\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
